In [112]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr
import google.generativeai as genai

In [113]:
load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY")

In [114]:
genai.configure(api_key=gemini_api_key)

In [115]:
context = {}

In [116]:
employees = glob.glob("knowledge-base/employees/*")

for employee in employees:
    name = employee.split(' ')[-1][:-3]
    doc = ""
    with open(employee, "r", encoding="utf-8") as f:
        doc = f.read()
    context[name]=doc

In [117]:
context.keys()

dict_keys(['Chen', 'Harper', 'Thomson', 'Lancaster', 'Carter', 'Tran', 'Blake', 'Bishop', 'Thompson', 'Spencer', 'Greene', 'Trenton'])

In [118]:
products = glob.glob("knowledge-base/products/*")

for product in products:
    name = product.split(os.sep)[-1][:-3]
    doc = ""
    with open(product, "r", encoding="utf-8") as f:
        doc = f.read()
    context[name]=doc

In [119]:
context.keys()

dict_keys(['Chen', 'Harper', 'Thomson', 'Lancaster', 'Carter', 'Tran', 'Blake', 'Bishop', 'Thompson', 'Spencer', 'Greene', 'Trenton', 'Carllm', 'Homellm', 'Markellm', 'Rellm'])

In [120]:
def get_relevant_context(message):
    relevant_context = []
    for context_title, context_details in context.items():
        if context_title.lower() in message.lower():
            relevant_context.append(context_details)
    return relevant_context

In [121]:
get_relevant_context("Who is lancaster?")

["# Avery Lancaster\n\n## Summary\n- **Date of Birth**: March 15, 1985  \n- **Job Title**: Co-Founder & Chief Executive Officer (CEO)  \n- **Location**: San Francisco, California  \n\n## Insurellm Career Progression\n- **2015 - Present**: Co-Founder & CEO  \n  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the company into the mainstream insurance market.  \n\n- **2013 - 2015**: Senior Product Manager at Innovate Insurance Solutions  \n  Before launching Insurellm, Avery was a leading Senior Product Manager at Innovate Insurance Solutions, where she developed groundbreaking insurance products aimed at the tech sector.  \n\n- **2010 - 2013**: Business Analyst at Edge Analytics  \n  Prior to joining Innovate, Avery worked as a Business Analyst, focusing on market trends and consumer preferen

In [122]:
def add_context(message):
    relevant_context = get_relevant_context(message)
    if relevant_context:
        message += "\n\nThe following additional context might be relevant in answering this question:\n\n"
        for relevant in relevant_context:
            message += relevant + "\n\n"
    return message

In [123]:
print(add_context("Who is Alex Lancaster?"))

Who is Alex Lancaster?

The following additional context might be relevant in answering this question:

# Avery Lancaster

## Summary
- **Date of Birth**: March 15, 1985  
- **Job Title**: Co-Founder & Chief Executive Officer (CEO)  
- **Location**: San Francisco, California  

## Insurellm Career Progression
- **2015 - Present**: Co-Founder & CEO  
  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the company into the mainstream insurance market.  

- **2013 - 2015**: Senior Product Manager at Innovate Insurance Solutions  
  Before launching Insurellm, Avery was a leading Senior Product Manager at Innovate Insurance Solutions, where she developed groundbreaking insurance products aimed at the tech sector.  

- **2010 - 2013**: Business Analyst at Edge Analytics  
  Prior to joining Innova

In [124]:
model = genai.GenerativeModel('gemini-2.0-flash')

def chat(message, history=[]):
    system_prompt = (
        "You are an expert in answering accurate questions about Insurellm, the Insurance Tech company. "
        "Give brief, accurate answers. If you don't know the answer, say so. Do not make anything up "
        "if you haven't been provided with relevant context."
    )

    chat_history_str = ""
    for user, bot in history:
        chat_history_str += f"User: {user}\nAssistant: {bot}\n"

    user_message = add_context(message)

    full_input = f"{system_prompt}\n\n{chat_history_str}User: {user_message}\nAssistant:"

    response = model.generate_content(full_input)
    answer = response.text

    history.append((message, answer))

    return answer, history


In [125]:
answer,chat_history = chat("Tell me about Alex Chen?")

In [126]:
answer

"Alex Chen is a Backend Software Engineer at Insurellm, located in San Francisco, California. He joined Insurellm in April 2020 as a Junior Backend Developer and was promoted to Backend Software Engineer in October 2021. He was later awarded the title of Senior Backend Software Engineer in March 2023. He has consistently received positive performance reviews and salary increases and is actively involved in Insurellm's Diversity & Inclusion initiatives.\n"

In [127]:
chat_history

[('Tell me about Alex Chen?',
  "Alex Chen is a Backend Software Engineer at Insurellm, located in San Francisco, California. He joined Insurellm in April 2020 as a Junior Backend Developer and was promoted to Backend Software Engineer in October 2021. He was later awarded the title of Senior Backend Software Engineer in March 2023. He has consistently received positive performance reviews and salary increases and is actively involved in Insurellm's Diversity & Inclusion initiatives.\n")]

In [128]:
answer,chat_history1 = chat("Tell me about Alex Chen's 2021 annual performance?", history=chat_history)

In [129]:
answer

'In 2021, Alex Chen achieved a 95% success rate in project delivery timelines and was awarded "Rising Star" at the annual company gala for outstanding contributions. His base salary increased to $90,000, and he received a performance bonus of $5,000.\n'

In [130]:
chat_history1

[('Tell me about Alex Chen?',
  "Alex Chen is a Backend Software Engineer at Insurellm, located in San Francisco, California. He joined Insurellm in April 2020 as a Junior Backend Developer and was promoted to Backend Software Engineer in October 2021. He was later awarded the title of Senior Backend Software Engineer in March 2023. He has consistently received positive performance reviews and salary increases and is actively involved in Insurellm's Diversity & Inclusion initiatives.\n"),
 ("Tell me about Alex Chen's 2021 annual performance?",
  'In 2021, Alex Chen achieved a 95% success rate in project delivery timelines and was awarded "Rising Star" at the annual company gala for outstanding contributions. His base salary increased to $90,000, and he received a performance bonus of $5,000.\n')]